# Setup

In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import time
import datetime
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import os
import shutil
import json
import pickle
import itertools
from scipy.signal import savgol_filter

np.random.seed(42)

from formats import experiment_pb2
from formats import  quantification_pb2

from skimage import io
import pandas as pd
import utils


from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

import torch
from torch.utils.data import Dataset, DataLoader, ConcatDataset, ChainDataset


import os
import pyro
import pyro.distributions as dist
import pyro.poutine
from pyro.infer import MCMC, NUTS
import math
import torch.nn as nn
import torch.nn.functional as F
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
from pyro.infer.autoguide.guides import AutoDiagonalNormal
import pyro.distributions.constraints as constraints
from tqdm import trange

import utils



In [2]:
local_radius_px = 5

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 


In [4]:
import data
dataset = data.get_dataset(local_radius_px=local_radius_px)
data_loader = torch.utils.data.DataLoader(dataset=dataset,batch_size=512)

In [5]:
from importlib import reload
import models
reload(models)
from models import FusionModel, SeparableVAE, JointVAE, NaiveFusionModel

In [6]:
def eval_recon(x_hat,q_hat, x, q):
    x_flat = x.reshape(-1,x.shape[-1])
    x_hat = x_hat.reshape(-1,x.shape[-1])

    x_mean = x_flat.mean(0)
    x_ssr = (x_flat - x_hat).pow(2).sum()
    x_sst = (x_flat - x_mean).pow(2).sum()
    x_r2 = 1 - x_ssr/x_sst

    q_mean = q.mean(0)
    q_ssr = (q - q_hat).pow(2).sum()
    q_sst = (q - q_mean).pow(2).sum()
    q_r2 = 1 - q_ssr/q_sst

    return x_r2.item(),q_r2.item()

In [7]:
def eval_model(z,c,m,h=64,d=16,r=5,dir='models'):
    model = torch.load(f'{dir}/{m.__name__}-z-{z}-c-{c}-h-{h}-d-{d}-r-{r}.pt').eval().to(device)    
    x , q = [], []
    # c, z = [], []
    x_hat, q_hat = [], []
    for batch_x,batch_q in tqdm(data_loader,total=1+(len(dataset)//512),leave=False):
        batch_x = batch_x.to(device)
        batch_q = batch_q.to(device)
        # batch_c, batch_z = model.encode(batch_x,batch_q)
        batch_x_hat,  batch_q_hat = model.reconstruct(batch_x,batch_q)
        x.append(batch_x.cpu())
        q.append(batch_q.cpu())
        # c.append(batch_c.cpu())
        # z.append(batch_z.cpu())
        x_hat.append(batch_x_hat.detach().cpu())
        q_hat.append(batch_q_hat.detach().cpu())

    x = torch.cat(x,dim=0).detach().cpu()
    q = torch.cat(q,dim=0).detach().cpu()
    # c = torch.cat(c,dim=0).detach().cpu()
    # z = torch.cat(z,dim=0).detach().cpu()
    x_hat = torch.cat(x_hat,dim=0).detach().cpu()
    q_hat = torch.cat(q_hat,dim=0).detach().cpu()

    return eval_recon(x_hat,q_hat,x, q)

In [11]:
data_loader = torch.utils.data.DataLoader(dataset=dataset,batch_size=512)
z_values = [1,2,3]
c_values = [1]
model_types = [FusionModel, JointVAE, NaiveFusionModel] 
eval_results= dict()
for model_type, z,c in tqdm(list(itertools.product(model_types, z_values,c_values))):
    model_label = f'{model_type.__name__}-z-{z}'
    eval_results[model_label] = eval_model(z,c,model_type,dir='models',h=64,d=16)
for k in eval_results:
    print(f'{k}     :   {eval_results[k]}')

100%|██████████| 9/9 [06:57<00:00, 46.35s/it]

FusionModel-z-1     :   (0.7524834871292114, 0.3602079153060913)
FusionModel-z-2     :   (0.9462422728538513, 0.9414879679679871)
FusionModel-z-3     :   (0.973613440990448, 0.9870474338531494)
JointVAE-z-1     :   (0.6342923641204834, 0.5894248485565186)
JointVAE-z-2     :   (0.7534908056259155, 0.7549608945846558)
JointVAE-z-3     :   (0.8179486989974976, 0.7550475597381592)
NaiveFusionModel-z-1     :   (0.8843056559562683, 0.6285606622695923)
NaiveFusionModel-z-2     :   (0.9404465556144714, 0.9001126289367676)
NaiveFusionModel-z-3     :   (0.984740138053894, 0.9292274713516235)


In [12]:
data_loader = torch.utils.data.DataLoader(dataset=dataset,batch_size=512)
z_values = [1,2,3]
c_values = [1]
model_types = [FusionModel, JointVAE, NaiveFusionModel] 
eval_results= dict()
for model_type, z,c in tqdm(list(itertools.product(model_types, z_values,c_values))):
    model_label = f'{model_type.__name__}-z-{z}'
    eval_results[model_label] = eval_model(z,c,model_type,dir='models',h=256,d=4)
for k in eval_results:
    print(f'{k}     :   {eval_results[k]}')

100%|██████████| 9/9 [06:30<00:00, 43.41s/it]

FusionModel-z-1     :   (0.8825869560241699, 0.9239256381988525)
FusionModel-z-2     :   (0.9650605320930481, 0.9836743474006653)
FusionModel-z-3     :   (0.9734076857566833, 0.9924401640892029)
JointVAE-z-1     :   (0.5275089740753174, 0.46742624044418335)
JointVAE-z-2     :   (0.8085591197013855, 0.8625675439834595)
JointVAE-z-3     :   (0.8366358280181885, 0.9295810461044312)
NaiveFusionModel-z-1     :   (0.892959713935852, 0.8058578968048096)
NaiveFusionModel-z-2     :   (0.9811556339263916, 0.754331111907959)
NaiveFusionModel-z-3     :   (0.9920786023139954, 0.8786454200744629)


In [25]:
eval_results= dict()
model_types = ['umap', 'umap-concat']
z_values = [2,3]
for model_type, z, in tqdm(list(itertools.product(model_types, z_values))):
    path = f'models/{model_type}-z-{z}.pkl'
    with open(path, 'rb') as f:
        umap_values = pickle.load(f)
    eval_results[f'{model_type}-{z}'] = eval_recon(torch.Tensor(umap_values['x_hat']),torch.Tensor(umap_values['q_hat']),torch.Tensor(umap_values['x']),torch.Tensor(umap_values['q']))

for k in eval_results:
    print(f'{k}     :   {eval_results[k]}')
    
    

100%|██████████| 4/4 [00:00<00:00, 204.39it/s]

umap-2     :   (0.013467013835906982, 0.6193883419036865)
umap-3     :   (0.6068850159645081, 0.3506179451942444)
umap-concat-2     :   (0.9585246443748474, 0.9579919576644897)
umap-concat-3     :   (0.9858996868133545, 0.9693862199783325)
